In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Flatten, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import backend as K
import numpy as np
import os
import random

# Set the seed for reproducibility
np.random.seed(0)

# Define the triplet loss function
# def triplet_loss(y_true, y_pred, alpha=0.2):
#     anchor, positive, negative = y_pred[:, 0], y_pred[:, 1], y_pred[:, 2]
#     pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
#     neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
#     loss = tf.maximum(pos_dist - neg_dist + alpha, 0.0)
#     return tf.reduce_mean(loss)
def triplet_loss(inputs, alpha=0.2):
    anchor, positive, negative = inputs['anchor'], inputs['positive'], inputs['negative']

    # Compute the distance between the anchor and positive embeddings
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)

    # Compute the distance between the anchor and negative embeddings
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)

    # Compute the triplet loss
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))

    return loss



# Define the base model
def get_base_model():
    return VGG16(include_top=False, input_shape=(224, 224, 3))

# Define the siamese network model
def get_siamese_model(base_model):
    # Add a global average pooling layer
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)

    # Define the input tensors
    anchor_input = Input(shape=(224, 224, 3), name='anchor_input')
    positive_input = Input(shape=(224, 224, 3), name='positive_input')
    negative_input = Input(shape=(224, 224, 3), name='negative_input')

    # Define new models with base_model input and x output
    encoder = Model(inputs=base_model.input, outputs=x)

    # Generate the encodings (output of the base model) for the anchor, positive, and negative images
    encoded_anchor = encoder(anchor_input)
    encoded_positive = encoder(positive_input)
    encoded_negative = encoder(negative_input)

    # Use Lambda layer to define custom layer for triplet loss
    loss_layer = Lambda(triplet_loss, output_shape=(1,), name='triplet_loss')({
        'anchor': encoded_anchor,
        'positive': encoded_positive,
        'negative': encoded_negative
    })

    # Define the siamese network model
    siamese_model = Model(inputs=[anchor_input, positive_input, negative_input], outputs=loss_layer)

    return siamese_model



# Compile the siamese network model
def compile_siamese_model(siamese_model):
    optimizer = Adam(lr=0.0001)
    siamese_model.compile(loss=triplet_loss, optimizer=optimizer)

# Define the training data generator
def get_train_data(train_dir):
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    # Get the image paths for each person
    person_paths = []
    for person_folder in os.listdir(train_dir):
        person_path = os.path.join(train_dir, person_folder)
        if os.path.isdir(person_path):
            person_paths.append(person_path)

    # Create pairs of anchor, positive, and negative images
    anchor_images, positive_images, negative_images = [], [], []
    for i in range(len(person_paths)):
        for j in range(i+1, len(person_paths)):
            person_i_images = os.listdir(person_paths[i])
            person_j_images = os.listdir(person_paths[j])
            for img_i in person_i_images:
                for img_j in person_j_images:
                    anchor_images.append(os.path.join(person_paths[i], img_i))
                    positive_images.append(os.path.join(person_paths[j], img_j))
                    negative_images.append(random.choice(person_paths[:i] + person_paths[i+1:j] + person_paths[j+1:]))

    # Convert image paths to arrays of images
    anchor_images = np.array([np.array(Image.open(img_path).resize((224, 224))) for img_path in anchor_images])
    positive_images = np.array([np.array(Image.open(img_path).resize((224, 224))) for img_path in positive_images])
    negative_images = np.array([np.array(Image.open(img_path).resize((224, 224))) for img_path in negative_images])

    # Create the data generator
    train_data = tf.data.Dataset.from_tensor_slices((anchor_images, positive_images, negative_images))
    train_data = train_data.batch(32)

    return train_data


# Train the siamese network model
def train_siamese_model(siamese_model, train_data):
    steps_per_epoch = train_data.samples // train_data.batch_size
    history = siamese_model.fit(train_data, steps_per_epoch=steps_per_epoch, epochs=10)
    return history


In [27]:
# Define the base model
base_model = get_base_model()

In [28]:
# Define the siamese network model
siamese_model = get_siamese_model(base_model)

In [29]:
# Compile the siamese network model
compile_siamese_model(siamese_model)

In [30]:

# Define the training dataset
train_dir = 'FaceRecognition\lfw'
train_data = get_train_data(train_dir)

In [ ]:
# Train the siamese network model
history = train_siamese_model(siamese_model, train_data)